In [1]:
import os 
import pickle
import pandas as pd 
import numpy as np 
from neuroHarmonize import harmonizationLearn, saveHarmonizationModel
from neuroHarmonize import harmonizationApply, loadHarmonizationModel

In [2]:
# out-of-sample ('oos') harmonization

# read in data
oos = pd.read_csv('/volume/projects/GL_Tesis/Analysis/site_harmonization/pronia_muse_t0_original_and_harmonized_combat.csv')

In [3]:
oos.columns

Index(['MUSE_Volume_701_neuroHarmonize', 'MUSE_Volume_601_neuroHarmonize',
       'MUSE_Volume_604_neuroHarmonize', 'MUSE_Volume_606_neuroHarmonize',
       'MUSE_Volume_607_neuroHarmonize', 'MUSE_Volume_613_neuroHarmonize',
       'MUSE_Volume_614_neuroHarmonize', 'MUSE_Volume_501_neuroHarmonize',
       'MUSE_Volume_502_neuroHarmonize', 'MUSE_Volume_503_neuroHarmonize',
       ...
       'PSN', 'SITE', 'SITE_idx', 'Age', 'ICV', 'Sex', 'Diagnosis',
       'Diagnosis_idx', 'Diagnosis_detailed', 'Sex0M1F'],
      dtype='object', length=528)

In [4]:
oos = oos.loc[:, oos.columns.str.endswith('neuroHarmonize') | oos.columns.isin(['ID', 'Age', 'SITE','Sex','Diagnosis_detailed','DLICV_baseline'])]

In [8]:
oos.head()

,MUSE_Volume_701_neuroHarmonize,MUSE_Volume_601_neuroHarmonize,MUSE_Volume_604_neuroHarmonize,MUSE_Volume_606_neuroHarmonize,MUSE_Volume_607_neuroHarmonize,MUSE_Volume_613_neuroHarmonize,MUSE_Volume_614_neuroHarmonize,MUSE_Volume_501_neuroHarmonize,MUSE_Volume_502_neuroHarmonize,MUSE_Volume_503_neuroHarmonize,...,MUSE_Volume_204_neuroHarmonize,MUSE_Volume_205_neuroHarmonize,MUSE_Volume_206_neuroHarmonize,MUSE_Volume_207_neuroHarmonize,ID,DLICV_baseline,SITE,Age,Sex,Diagnosis_detailed
0,1.134341e+06,631167.484465,464659.406398,312093.590887,227027.441050,310490.079201,226986.504554,10626.699239,127232.502211,57907.754487,...,3456.585434,2730.869454,1117.909658,1298.385779,100427,1.326045e+06,Uni Udine,28.408219,2,HC
1,1.222849e+06,694956.509578,494825.453684,345818.363519,241032.872676,339444.487093,241635.455041,12171.574727,167486.031437,59258.645977,...,4311.004963,5156.539135,1716.895225,2210.340718,100742,1.413257e+06,Uni Udine,23.934247,2,HC
2,1.458366e+06,820431.561996,595392.557273,403556.297119,288958.368442,406019.252486,293613.404009,12702.653155,179256.978828,68469.657516,...,4728.331840,4631.517245,2206.557209,2694.924888,100861,1.674748e+06,UBS,30.375342,1,HC
3,1.395021e+06,836333.878866,524476.464961,410691.491630,255984.833998,415172.260032,256808.669512,11668.905268,172883.481115,66079.371216,...,3802.660817,4067.351856,1737.593084,1890.478543,100963,1.607498e+06,UBS,16.583562,1,HC
4,1.297730e+06,774271.757093,490572.326161,382448.449800,238618.487929,381711.431014,241119.220706,10779.896573,151511.067873,60782.451612,...,3181.244087,3486.917922,1622.865004,1900.121344,101712,1.495299e+06,Uni Udine,21.117808,1,HC


In [9]:
oos.columns

Index(['MUSE_Volume_701_neuroHarmonize', 'MUSE_Volume_601_neuroHarmonize',
       'MUSE_Volume_604_neuroHarmonize', 'MUSE_Volume_606_neuroHarmonize',
       'MUSE_Volume_607_neuroHarmonize', 'MUSE_Volume_613_neuroHarmonize',
       'MUSE_Volume_614_neuroHarmonize', 'MUSE_Volume_501_neuroHarmonize',
       'MUSE_Volume_502_neuroHarmonize', 'MUSE_Volume_503_neuroHarmonize',
       ...
       'MUSE_Volume_204_neuroHarmonize', 'MUSE_Volume_205_neuroHarmonize',
       'MUSE_Volume_206_neuroHarmonize', 'MUSE_Volume_207_neuroHarmonize',
       'ID', 'DLICV_baseline', 'SITE', 'Age', 'Sex', 'Diagnosis_detailed'],
      dtype='object', length=264)

In [10]:
oos['SITE'] 

0       Uni Udine
1       Uni Udine
2             UBS
3             UBS
4       Uni Udine
          ...    
1431          UBS
1432          LMU
1433    Uni Udine
1434          LMU
1435          LMU
Name: SITE, Length: 1436, dtype: object

In [11]:
unique_values = oos['SITE'].unique()
print(unique_values)


['Uni Udine' 'UBS' 'LMU' 'Uni Turku' 'UKK' 'Uni BHAM' 'MilanNig' 'UMUENS'
 'UBARI' 'UDUS']


In [12]:
# check min and max ages of data to make sure they are between 15 & 50, and remove subtjects outside of range 
oos['Age'].min() #16.0383561643836
oos['Age'].max() #41.0301369863014

41.0301369863014

In [13]:
oos['ID']

0       100427
1       100742
2       100861
3       100963
4       101712
         ...  
1431     99035
1432     99095
1433      9988
1434     99986
1435       999
Name: ID, Length: 1436, dtype: int64

In [14]:
oos['Sex']

0       2
1       2
2       1
3       1
4       1
       ..
1431    1
1432    2
1433    1
1434    2
1435    1
Name: Sex, Length: 1436, dtype: int64

In [15]:
oos["Sex0M1F"] = oos["Sex"] - 1

In [16]:
oos['Sex0M1F']

0       1
1       1
2       0
3       0
4       0
       ..
1431    0
1432    1
1433    0
1434    1
1435    0
Name: Sex0M1F, Length: 1436, dtype: int64

In [17]:
oos['Original_Site'] = oos['SITE']
oos['SITE'] = 'Pronia'

In [18]:
unique_values = oos['SITE'].unique()
print(unique_values)

['Pronia']


In [19]:
oos['Group'] = oos['Diagnosis_detailed']

In [20]:
# recode Diagnosis variable: patients = 1, controls = 0
oos['Diagnosis'] = oos['Diagnosis_detailed'].apply(lambda x: 1 if x != 'HC' else 0)

In [21]:
oos['Diagnosis'] 

0       0
1       0
2       0
3       0
4       0
       ..
1431    1
1432    1
1433    1
1434    1
1435    1
Name: Diagnosis, Length: 1436, dtype: int64

In [22]:
oos['Group']

0        HC
1        HC
2        HC
3        HC
4        HC
       ... 
1431    CHR
1432    ROP
1433    ROP
1434    ROP
1435    ROP
Name: Group, Length: 1436, dtype: object

In [23]:
# isolate controls 
hc = oos[oos.Group=="HC"]

# isolate the ROI columns & convert to array


In [24]:
hc.head()

,MUSE_Volume_701_neuroHarmonize,MUSE_Volume_601_neuroHarmonize,MUSE_Volume_604_neuroHarmonize,MUSE_Volume_606_neuroHarmonize,MUSE_Volume_607_neuroHarmonize,MUSE_Volume_613_neuroHarmonize,MUSE_Volume_614_neuroHarmonize,MUSE_Volume_501_neuroHarmonize,MUSE_Volume_502_neuroHarmonize,MUSE_Volume_503_neuroHarmonize,...,ID,DLICV_baseline,SITE,Age,Sex,Diagnosis_detailed,Sex0M1F,Original_Site,Group,Diagnosis
0,1.134341e+06,631167.484465,464659.406398,312093.590887,227027.441050,310490.079201,226986.504554,10626.699239,127232.502211,57907.754487,...,100427,1.326045e+06,Pronia,28.408219,2,HC,1,Uni Udine,HC,0
1,1.222849e+06,694956.509578,494825.453684,345818.363519,241032.872676,339444.487093,241635.455041,12171.574727,167486.031437,59258.645977,...,100742,1.413257e+06,Pronia,23.934247,2,HC,1,Uni Udine,HC,0
2,1.458366e+06,820431.561996,595392.557273,403556.297119,288958.368442,406019.252486,293613.404009,12702.653155,179256.978828,68469.657516,...,100861,1.674748e+06,Pronia,30.375342,1,HC,0,UBS,HC,0
3,1.395021e+06,836333.878866,524476.464961,410691.491630,255984.833998,415172.260032,256808.669512,11668.905268,172883.481115,66079.371216,...,100963,1.607498e+06,Pronia,16.583562,1,HC,0,UBS,HC,0
4,1.297730e+06,774271.757093,490572.326161,382448.449800,238618.487929,381711.431014,241119.220706,10779.896573,151511.067873,60782.451612,...,101712,1.495299e+06,Pronia,21.117808,1,HC,0,Uni Udine,HC,0


In [25]:
hc.columns

Index(['MUSE_Volume_701_neuroHarmonize', 'MUSE_Volume_601_neuroHarmonize',
       'MUSE_Volume_604_neuroHarmonize', 'MUSE_Volume_606_neuroHarmonize',
       'MUSE_Volume_607_neuroHarmonize', 'MUSE_Volume_613_neuroHarmonize',
       'MUSE_Volume_614_neuroHarmonize', 'MUSE_Volume_501_neuroHarmonize',
       'MUSE_Volume_502_neuroHarmonize', 'MUSE_Volume_503_neuroHarmonize',
       ...
       'ID', 'DLICV_baseline', 'SITE', 'Age', 'Sex', 'Diagnosis_detailed',
       'Sex0M1F', 'Original_Site', 'Group', 'Diagnosis'],
      dtype='object', length=268)

In [26]:
hcrois = hc.iloc[:, 0:258]
hcrois

,MUSE_Volume_701_neuroHarmonize,MUSE_Volume_601_neuroHarmonize,MUSE_Volume_604_neuroHarmonize,MUSE_Volume_606_neuroHarmonize,MUSE_Volume_607_neuroHarmonize,MUSE_Volume_613_neuroHarmonize,MUSE_Volume_614_neuroHarmonize,MUSE_Volume_501_neuroHarmonize,MUSE_Volume_502_neuroHarmonize,MUSE_Volume_503_neuroHarmonize,...,MUSE_Volume_198_neuroHarmonize,MUSE_Volume_199_neuroHarmonize,MUSE_Volume_200_neuroHarmonize,MUSE_Volume_201_neuroHarmonize,MUSE_Volume_202_neuroHarmonize,MUSE_Volume_203_neuroHarmonize,MUSE_Volume_204_neuroHarmonize,MUSE_Volume_205_neuroHarmonize,MUSE_Volume_206_neuroHarmonize,MUSE_Volume_207_neuroHarmonize
0,1.134341e+06,631167.484465,464659.406398,312093.590887,227027.441050,310490.079201,226986.504554,10626.699239,127232.502211,57907.754487,...,7330.575496,8510.078321,5603.272433,6958.758332,7951.825640,8165.389357,3456.585434,2730.869454,1117.909658,1298.385779
1,1.222849e+06,694956.509578,494825.453684,345818.363519,241032.872676,339444.487093,241635.455041,12171.574727,167486.031437,59258.645977,...,8285.066639,8931.696580,6256.756451,7270.107141,8926.336696,8988.773011,4311.004963,5156.539135,1716.895225,2210.340718
2,1.458366e+06,820431.561996,595392.557273,403556.297119,288958.368442,406019.252486,293613.404009,12702.653155,179256.978828,68469.657516,...,11201.950076,10253.315258,9285.529947,8013.128502,11112.614835,11291.892626,4728.331840,4631.517245,2206.557209,2694.924888
3,1.395021e+06,836333.878866,524476.464961,410691.491630,255984.833998,415172.260032,256808.669512,11668.905268,172883.481115,66079.371216,...,13757.781122,12783.103262,7962.664866,8312.895361,12470.112603,11132.290177,3802.660817,4067.351856,1737.593084,1890.478543
4,1.297730e+06,774271.757093,490572.326161,382448.449800,238618.487929,381711.431014,241119.220706,10779.896573,151511.067873,60782.451612,...,11614.743900,12054.210009,8450.392757,7874.966960,11128.530040,10087.124495,3181.244087,3486.917922,1622.865004,1900.121344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1.113717e+06,661783.157474,424220.711455,328304.455523,208724.859300,324898.012325,205845.725297,9686.163523,130194.551412,53707.722880,...,8286.436506,9559.419933,7502.508241,6986.704131,9769.384935,12310.619565,3012.112705,3098.622373,1029.753591,745.088693
453,1.075012e+06,616997.248165,420555.075026,304415.683199,207156.064738,304703.383310,203155.040797,10273.480814,136654.728790,53664.523087,...,8585.134541,8167.077522,5974.847879,6409.631051,7347.068259,6665.491107,3758.587790,3122.564070,1128.922069,1266.933629
454,1.321452e+06,763302.497796,482987.311010,378461.673469,236907.626989,376465.780723,234825.472567,11149.840203,127916.479518,65562.554507,...,13387.055712,12832.459896,7483.367851,8361.729039,10877.311843,11818.197155,3269.210556,3336.670809,1675.891383,1538.216617
455,1.082870e+06,597129.429673,459400.265140,294502.341173,224033.643490,293719.097231,222256.519432,12998.293226,129837.531861,53127.617347,...,8074.955162,6766.157716,5730.351714,6935.513650,7026.819891,6888.865250,2893.856128,2876.428627,1568.983496,1523.409412


In [27]:
hcrois = np.array(hcrois)

In [28]:
# get HC covariates in separate variable 
covars = hc[['SITE', 'Age', 'Sex0M1F', 'DLICV_baseline']]
covars = covars.reset_index()
covars = covars.drop(['index'], axis=1)

In [30]:

# load model i sent you
model = loadHarmonizationModel('/volume/projects/GL_Tesis/Analysis/secondary_harmonization/harmonize_model_chronicsonly_1550age.pkl')

# next step adds the sites from the out-of-sample data to the model (aug=augmented) 
model_aug, _ = harmonizationLearn(hcrois, covars, smooth_terms=['Age'], smooth_term_bounds=(np.floor(np.min(oos.Age)), np.ceil(np.max(oos.Age))), orig_model=model, seed=30323) # today's date
# check if it worked: the site labels of the model should now also include the oos sites
model_aug['SITE_labels'] 

# save the augmented model 
saveHarmonizationModel(model_aug, 'model_with_outofsample_sites_added.pkl')




[neuroHarmonize]: Saving model object, estimated size in MB: 0.21


In [31]:
model_aug['SITE_labels'] 

array(['Enigma', 'Melbourne', 'Munich', 'Peking', 'Tianji',
       'UMC_Utrecht_ffe', 'Utrecht', 'Pronia'], dtype=object)

In [32]:
oos.head()

,MUSE_Volume_701_neuroHarmonize,MUSE_Volume_601_neuroHarmonize,MUSE_Volume_604_neuroHarmonize,MUSE_Volume_606_neuroHarmonize,MUSE_Volume_607_neuroHarmonize,MUSE_Volume_613_neuroHarmonize,MUSE_Volume_614_neuroHarmonize,MUSE_Volume_501_neuroHarmonize,MUSE_Volume_502_neuroHarmonize,MUSE_Volume_503_neuroHarmonize,...,ID,DLICV_baseline,SITE,Age,Sex,Diagnosis_detailed,Sex0M1F,Original_Site,Group,Diagnosis
0,1.134341e+06,631167.484465,464659.406398,312093.590887,227027.441050,310490.079201,226986.504554,10626.699239,127232.502211,57907.754487,...,100427,1.326045e+06,Pronia,28.408219,2,HC,1,Uni Udine,HC,0
1,1.222849e+06,694956.509578,494825.453684,345818.363519,241032.872676,339444.487093,241635.455041,12171.574727,167486.031437,59258.645977,...,100742,1.413257e+06,Pronia,23.934247,2,HC,1,Uni Udine,HC,0
2,1.458366e+06,820431.561996,595392.557273,403556.297119,288958.368442,406019.252486,293613.404009,12702.653155,179256.978828,68469.657516,...,100861,1.674748e+06,Pronia,30.375342,1,HC,0,UBS,HC,0
3,1.395021e+06,836333.878866,524476.464961,410691.491630,255984.833998,415172.260032,256808.669512,11668.905268,172883.481115,66079.371216,...,100963,1.607498e+06,Pronia,16.583562,1,HC,0,UBS,HC,0
4,1.297730e+06,774271.757093,490572.326161,382448.449800,238618.487929,381711.431014,241119.220706,10779.896573,151511.067873,60782.451612,...,101712,1.495299e+06,Pronia,21.117808,1,HC,0,Uni Udine,HC,0


In [36]:
oosrois = oos.iloc[:, 0:258] # isolate rois
oosrois


,MUSE_Volume_701_neuroHarmonize,MUSE_Volume_601_neuroHarmonize,MUSE_Volume_604_neuroHarmonize,MUSE_Volume_606_neuroHarmonize,MUSE_Volume_607_neuroHarmonize,MUSE_Volume_613_neuroHarmonize,MUSE_Volume_614_neuroHarmonize,MUSE_Volume_501_neuroHarmonize,MUSE_Volume_502_neuroHarmonize,MUSE_Volume_503_neuroHarmonize,...,MUSE_Volume_198_neuroHarmonize,MUSE_Volume_199_neuroHarmonize,MUSE_Volume_200_neuroHarmonize,MUSE_Volume_201_neuroHarmonize,MUSE_Volume_202_neuroHarmonize,MUSE_Volume_203_neuroHarmonize,MUSE_Volume_204_neuroHarmonize,MUSE_Volume_205_neuroHarmonize,MUSE_Volume_206_neuroHarmonize,MUSE_Volume_207_neuroHarmonize
0,1.134341e+06,631167.484465,464659.406398,312093.590887,227027.441050,310490.079201,226986.504554,10626.699239,127232.502211,57907.754487,...,7330.575496,8510.078321,5603.272433,6958.758332,7951.825640,8165.389357,3456.585434,2730.869454,1117.909658,1298.385779
1,1.222849e+06,694956.509578,494825.453684,345818.363519,241032.872676,339444.487093,241635.455041,12171.574727,167486.031437,59258.645977,...,8285.066639,8931.696580,6256.756451,7270.107141,8926.336696,8988.773011,4311.004963,5156.539135,1716.895225,2210.340718
2,1.458366e+06,820431.561996,595392.557273,403556.297119,288958.368442,406019.252486,293613.404009,12702.653155,179256.978828,68469.657516,...,11201.950076,10253.315258,9285.529947,8013.128502,11112.614835,11291.892626,4728.331840,4631.517245,2206.557209,2694.924888
3,1.395021e+06,836333.878866,524476.464961,410691.491630,255984.833998,415172.260032,256808.669512,11668.905268,172883.481115,66079.371216,...,13757.781122,12783.103262,7962.664866,8312.895361,12470.112603,11132.290177,3802.660817,4067.351856,1737.593084,1890.478543
4,1.297730e+06,774271.757093,490572.326161,382448.449800,238618.487929,381711.431014,241119.220706,10779.896573,151511.067873,60782.451612,...,11614.743900,12054.210009,8450.392757,7874.966960,11128.530040,10087.124495,3181.244087,3486.917922,1622.865004,1900.121344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,1.333362e+06,796239.602134,501092.637314,393314.124677,243407.409395,391756.098556,245998.379595,11667.535253,160626.592935,60893.557024,...,11855.734250,11411.302675,8380.646121,10014.692485,12097.052519,10402.651005,4190.074698,4434.229766,1863.190561,1836.675050
1432,1.146346e+06,621528.706406,484034.739836,303393.862857,230058.417309,308768.913488,241383.005038,12617.366564,128761.146280,55092.963921,...,7575.638453,8196.352250,6277.633796,5940.330121,9421.355338,8447.103208,2353.296149,2994.706617,1465.929413,1101.988976
1433,1.221237e+06,729266.180855,460178.113967,360162.862486,223873.254523,358801.152517,224709.071663,11556.618660,153312.166607,61154.518166,...,9882.971340,8367.267893,6642.658493,7981.294250,10139.055734,10405.762157,2644.695068,4154.189808,1770.247133,1895.881602
1434,1.195990e+06,666592.726632,477549.008542,326933.374899,232351.969773,330623.352735,233345.817456,11826.667711,135334.503460,60953.677628,...,9603.193312,7908.643426,7672.513512,7058.912729,8526.278602,8398.550821,3582.385749,4155.171301,625.720562,1399.084964


In [38]:
roinames = oosrois.columns #all 258 ROIs
roinames

Index(['MUSE_Volume_701_neuroHarmonize', 'MUSE_Volume_601_neuroHarmonize',
       'MUSE_Volume_604_neuroHarmonize', 'MUSE_Volume_606_neuroHarmonize',
       'MUSE_Volume_607_neuroHarmonize', 'MUSE_Volume_613_neuroHarmonize',
       'MUSE_Volume_614_neuroHarmonize', 'MUSE_Volume_501_neuroHarmonize',
       'MUSE_Volume_502_neuroHarmonize', 'MUSE_Volume_503_neuroHarmonize',
       ...
       'MUSE_Volume_198_neuroHarmonize', 'MUSE_Volume_199_neuroHarmonize',
       'MUSE_Volume_200_neuroHarmonize', 'MUSE_Volume_201_neuroHarmonize',
       'MUSE_Volume_202_neuroHarmonize', 'MUSE_Volume_203_neuroHarmonize',
       'MUSE_Volume_204_neuroHarmonize', 'MUSE_Volume_205_neuroHarmonize',
       'MUSE_Volume_206_neuroHarmonize', 'MUSE_Volume_207_neuroHarmonize'],
      dtype='object', length=258)

In [39]:
oosrois = np.array(oosrois) # convert to array 

In [40]:
# apply new augmented model to full dataset (including controls again)

# apply harmonization to full sample
harmonized_oosrois = harmonizationApply(oosrois, oos[['SITE', 'Age', 'Sex0M1F', 'DLICV_baseline']], model_aug)

# combine original and harmonized rois and save 
hoos = pd.concat([oos.reset_index(), pd.DataFrame(harmonized_oosrois, columns=['H_' + s for s in roinames])], axis=1)


In [41]:
hoos

,index,MUSE_Volume_701_neuroHarmonize,MUSE_Volume_601_neuroHarmonize,MUSE_Volume_604_neuroHarmonize,MUSE_Volume_606_neuroHarmonize,MUSE_Volume_607_neuroHarmonize,MUSE_Volume_613_neuroHarmonize,MUSE_Volume_614_neuroHarmonize,MUSE_Volume_501_neuroHarmonize,MUSE_Volume_502_neuroHarmonize,...,H_MUSE_Volume_198_neuroHarmonize,H_MUSE_Volume_199_neuroHarmonize,H_MUSE_Volume_200_neuroHarmonize,H_MUSE_Volume_201_neuroHarmonize,H_MUSE_Volume_202_neuroHarmonize,H_MUSE_Volume_203_neuroHarmonize,H_MUSE_Volume_204_neuroHarmonize,H_MUSE_Volume_205_neuroHarmonize,H_MUSE_Volume_206_neuroHarmonize,H_MUSE_Volume_207_neuroHarmonize
0,0,1.134341e+06,631167.484465,464659.406398,312093.590887,227027.441050,310490.079201,226986.504554,10626.699239,127232.502211,...,7631.098681,8879.041538,5976.030039,7393.997417,8542.939077,8606.530867,3632.120795,2864.811406,1101.958492,1367.212541
1,1,1.222849e+06,694956.509578,494825.453684,345818.363519,241032.872676,339444.487093,241635.455041,12171.574727,167486.031437,...,8588.389706,9289.637765,6627.713077,7684.748079,9544.366968,9437.068518,4493.564336,5279.033987,1732.613861,2389.834117
2,2,1.458366e+06,820431.561996,595392.557273,403556.297119,288958.368442,406019.252486,293613.404009,12702.653155,179256.978828,...,11534.474882,10617.131814,9540.020586,8419.611997,11762.548495,11750.989925,4910.678421,4758.422399,2238.951090,2893.888194
3,3,1.395021e+06,836333.878866,524476.464961,410691.491630,255984.833998,415172.260032,256808.669512,11668.905268,172883.481115,...,14125.021021,13208.548108,8297.593502,8712.794793,13250.570948,11579.721858,3972.200831,4198.158764,1741.521441,1975.952896
4,4,1.297730e+06,774271.757093,490572.326161,382448.449800,238618.487929,381711.431014,241119.220706,10779.896573,151511.067873,...,11962.613445,12490.148699,8709.416069,8295.496933,11864.633264,10541.607350,3347.250395,3618.717793,1629.056171,2013.210993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,1431,1.333362e+06,796239.602134,501092.637314,393314.124677,243407.409395,391756.098556,245998.379595,11667.535253,160626.592935,...,12199.682235,11812.428109,8661.786418,10524.047582,12891.850039,10847.483168,4366.571467,4561.821386,1881.875115,1931.073852
1432,1432,1.146346e+06,621528.706406,484034.739836,303393.862857,230058.417309,308768.913488,241383.005038,12617.366564,128761.146280,...,7884.101895,8559.532483,6601.881143,6330.323368,10146.880475,8900.170309,2517.181989,3126.201372,1473.434879,1145.946965
1433,1433,1.221237e+06,729266.180855,460178.113967,360162.862486,223873.254523,358801.152517,224709.071663,11556.618660,153312.166607,...,10201.125227,8691.011422,6999.282209,8405.348060,10812.361577,10862.654699,2804.525309,4282.185269,1789.722967,2016.208904
1434,1434,1.195990e+06,666592.726632,477549.008542,326933.374899,232351.969773,330623.352735,233345.817456,11826.667711,135334.503460,...,9919.462137,8218.125255,7978.859493,7445.409443,9080.534207,8815.262095,3754.040249,4284.229373,565.746014,1450.593159


In [42]:
hoos_combat = pd.DataFrame(harmonized_oosrois, columns=['H_' + s for s in roinames])

In [43]:
hoos_combat

,H_MUSE_Volume_701_neuroHarmonize,H_MUSE_Volume_601_neuroHarmonize,H_MUSE_Volume_604_neuroHarmonize,H_MUSE_Volume_606_neuroHarmonize,H_MUSE_Volume_607_neuroHarmonize,H_MUSE_Volume_613_neuroHarmonize,H_MUSE_Volume_614_neuroHarmonize,H_MUSE_Volume_501_neuroHarmonize,H_MUSE_Volume_502_neuroHarmonize,H_MUSE_Volume_503_neuroHarmonize,...,H_MUSE_Volume_198_neuroHarmonize,H_MUSE_Volume_199_neuroHarmonize,H_MUSE_Volume_200_neuroHarmonize,H_MUSE_Volume_201_neuroHarmonize,H_MUSE_Volume_202_neuroHarmonize,H_MUSE_Volume_203_neuroHarmonize,H_MUSE_Volume_204_neuroHarmonize,H_MUSE_Volume_205_neuroHarmonize,H_MUSE_Volume_206_neuroHarmonize,H_MUSE_Volume_207_neuroHarmonize
0,1.186514e+06,655681.221219,493854.873996,324386.043981,241793.297156,322727.301347,241455.033025,10625.481043,130047.749279,58217.466881,...,7631.098681,8879.041538,5976.030039,7393.997417,8542.939077,8606.530867,3632.120795,2864.811406,1101.958492,1367.212541
1,1.275069e+06,719790.278670,523988.432011,358323.198671,255816.525218,351743.220189,256077.835805,12170.355139,172116.393715,59425.166156,...,8588.389706,9289.637765,6627.713077,7684.748079,9544.366968,9437.068518,4493.564336,5279.033987,1732.613861,2389.834117
2,1.510642e+06,845952.924317,624628.601820,416257.967713,303694.401158,418741.404850,308092.398807,12701.435760,183409.191510,68750.441220,...,11534.474882,10617.131814,9540.020586,8419.611997,11762.548495,11750.989925,4910.678421,4758.422399,2238.951090,2893.888194
3,1.447158e+06,862001.509764,553788.130202,423432.626590,270691.163637,427968.489878,271376.614506,11667.687382,176806.732294,66258.551783,...,14125.021021,13208.548108,8297.593502,8712.794793,13250.570948,11579.721858,3972.200831,4198.158764,1741.521441,1975.952896
4,1.350002e+06,800110.753581,519856.163865,395352.792492,253339.479308,394527.625611,255656.820204,10778.678757,154703.158241,60843.193736,...,11962.613445,12490.148699,8709.416069,8295.496933,11864.633264,10541.607350,3347.250395,3618.717793,1629.056171,2013.210993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,1.385575e+06,821895.226597,530362.387618,406127.533211,258129.089423,404453.759713,260523.947253,11666.316703,164168.041966,60861.599050,...,12199.682235,11812.428109,8661.786418,10524.047582,12891.850039,10847.483168,4366.571467,4561.821386,1881.875115,1931.073852
1432,1.198567e+06,646163.183431,513212.971254,315620.044189,244801.716802,321155.410660,255789.857334,12616.146350,131711.810432,55223.343071,...,7884.101895,8559.532483,6601.881143,6330.323368,10146.880475,8900.170309,2517.181989,3126.201372,1473.434879,1145.946965
1433,1.273243e+06,754002.623666,489461.533560,372544.897574,238590.213034,371104.388827,239259.138438,11555.398864,156756.301142,61371.482245,...,10201.125227,8691.011422,6999.282209,8405.348060,10812.361577,10862.654699,2804.525309,4282.185269,1789.722967,2016.208904
1434,1.247805e+06,689499.672238,506814.020761,338379.348170,247080.954492,342234.648942,247876.648132,11825.448765,138102.227462,61149.351311,...,9919.462137,8218.125255,7978.859493,7445.409443,9080.534207,8815.262095,3754.040249,4284.229373,565.746014,1450.593159


In [44]:
hoos.iloc[:, -5:]


,H_MUSE_Volume_203_neuroHarmonize,H_MUSE_Volume_204_neuroHarmonize,H_MUSE_Volume_205_neuroHarmonize,H_MUSE_Volume_206_neuroHarmonize,H_MUSE_Volume_207_neuroHarmonize
0,8606.530867,3632.120795,2864.811406,1101.958492,1367.212541
1,9437.068518,4493.564336,5279.033987,1732.613861,2389.834117
2,11750.989925,4910.678421,4758.422399,2238.951090,2893.888194
3,11579.721858,3972.200831,4198.158764,1741.521441,1975.952896
4,10541.607350,3347.250395,3618.717793,1629.056171,2013.210993
...,...,...,...,...,...
1431,10847.483168,4366.571467,4561.821386,1881.875115,1931.073852
1432,8900.170309,2517.181989,3126.201372,1473.434879,1145.946965
1433,10862.654699,2804.525309,4282.185269,1789.722967,2016.208904
1434,8815.262095,3754.040249,4284.229373,565.746014,1450.593159


In [45]:
hoos.to_csv('/volume/projects/GL_Tesis/Analysis/secondary_harmonization/Original_and_Harmonized_ROIS_outofsample_sites_new.csv', index = False, encoding = 'utf-8-sig')

In [46]:
hoos_combat.to_csv('/volume/projects/GL_Tesis/Analysis/secondary_harmonization/Harmonized_only_ROIS_outofsample_sites_new.csv', index = False, encoding = 'utf-8-sig')

In [50]:
harmonized_oosrois[]

array([[1484448.93479651,  824384.6867998 ,  600559.61659433, ...,
          10923.36565584,  150071.34978553,   69180.73201447],
       [1139175.24834356,  641504.57036982,  459041.57806288, ...,
          10711.95321405,  131332.33562026,   54421.84814265],
       [1398563.69569854,  769898.57138818,  588574.91269122, ...,
          11042.46517369,  162749.10269252,   63789.49567488],
       ...,
       [1227289.71734562,  742800.61348607,  451120.88464927, ...,
          11459.17526389,  155785.56318206,   53452.66365529],
       [1259188.08515681,  727230.35382034,  490808.13602035, ...,
          12513.59110317,  156128.30641835,   58026.66300156],
       [1402294.79100886,  800060.7841387 ,  560063.36863698, ...,
          13859.29130261,  157609.28374567,   70083.82791507]])